In [1]:
from zoomtools import solr, transform, munge
import pandas as pd
ss6 = solr.conn()

http://solarstage06.xoominfo.com:8983/solr/zoomCollection


In [11]:
args = {'fl':'c_id,d_company,url,d_company_ext_names_xml,annotations',
        'sort':'random_2345 asc',
        'rows':1000}

df = solr.query(ss6, 'emp:4999 -c_class:cpycode.inc5000 -sector', args)
df.head(1)

,c_id,d_company,url,d_company_ext_names_xml,annotations
0,372139962,"China Datang Corporation Renewable Power Co., ...",www.dtxny.com.cn,[<?xml version='1.0' encoding='UTF-8'?><name c...,"[datainfusion.lexisnexis, datainfusion.willow]"


In [12]:
xmls = transform.one2many_to_one2one(df['c_id'], df['d_company_ext_names_xml'])
xmls.head(1)

,key,value
0,372139962,<?xml version='1.0' encoding='UTF-8'?><name co...


In [13]:
xmls['value_enc'] = xmls['value'].map(lambda string: string.encode('utf8'))
parsed = transform.xml_tag_to_dataframe(xmls['key'], xmls['value_enc'], "//name")
parsed.head(1)

,count,id,name,normalizedName,value
0,24,372139962,None,proc datang corporation renewable power,"China Datang Corporation Renewable Power Co., ..."


In [14]:
anots = pd.DataFrame(df['c_id']).join(pd.DataFrame(df['annotations']).join(pd.DataFrame(df['d_company'])))
parsed1 = parsed.merge(anots, left_on ='id', right_on='c_id')

In [15]:
parsed1.to_csv("ext_names2.csv", index=False)